In [2]:
import os
os.environ["LOKY_MAX_CPU_COUNT"] = "20"  # ← 改成你的物理核数
os.environ["OMP_NUM_THREADS"] = "1"

import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from utils2 import *

np.random.seed(47)

# 加载原始数据 
load_data   = np.load('user_load.npz')
PV_data     = np.load('PV_data.npz')
Radiation   = np.load('Radiation.npz')

load_data   = load_data['data']
PV_data     = PV_data['PV']
Radiation   = Radiation['Radiation'][:365*48]

num_user    = 1000

# 保证Prosumer数据不会出现负数
load_data  += 1

# 放缩光伏功率数据
x_min = np.min(PV_data, axis=1, keepdims=True)  # 每行最小值
x_max = np.max(PV_data, axis=1, keepdims=True)  # 每行最大值
PV_data = 1.05 * (PV_data - x_min) / (x_max - x_min)

# 处理Prosumer和Consumer数据
user_list = np.arange(num_user)
site_list = np.arange(int(len(user_list)*0.1))
prosumer_list = np.random.choice(user_list, size=int(len(user_list)*0.1), replace=False)
consumer_list = np.setdiff1d(user_list, prosumer_list)

load_data   = load_data[user_list]
PV_data     = PV_data[site_list][:,:365*48]
load_data[prosumer_list] -= PV_data
load_data = np.clip(load_data, 0, None)


# 生成时间序列（从 2010-01-01 开始）
date_rng = pd.date_range('2010-01-01', periods=365*24, freq='60min')

# 提取 weekday（0=Mon, 6=Sun）
weekday = date_rng.weekday.values  # shape: (17520,)

# One-hot 编码 (7 类)
weekday_onehot = np.eye(7)[weekday]  # shape: (17520, 7)

# 去掉一列以消除共线性（例如去掉 Sunday 列）
weekday_onehot = weekday_onehot[:, 1:]  # shape: (17520, 6)

# 计算每个时间步对应的小时数
# 0, 0.5, 1.0, ... , 23.5, 然后循环
hours = (np.arange(365*24) % 24)

# 生成正弦/余弦特征（24小时周期）
hour_sin = np.sin(2 * np.pi * hours / 24).reshape(-1,1)
hour_cos = np.cos(2 * np.pi * hours / 24).reshape(-1,1)

Calendar = np.concatenate((weekday_onehot, hour_sin), axis=1)
Calendar = np.concatenate((Calendar, hour_cos), axis=1)

print(Calendar.shape)
load_data = load_data.reshape(num_user, 2,-1).sum(axis=1)
Radiation = Radiation.reshape(2,-1).sum(axis=0)
print(load_data.shape)

(8760, 8)
(1000, 8760)


In [3]:
K           = 20          # 簇数 Num of cluster

train_ratio = 0.72
val_ratio   = 0.08
max_epoch   = 100
save_path   = "Process_3\\CLC\\"
os.makedirs(save_path,exist_ok=True)
MAE_list = []
MAPE_list = []
# 4/5

process_list = []
for K in range(15,16):
    # 1.初始随机分配 Initialization
    kmeans = KMeans(n_clusters=K,random_state=47,n_init='auto')
    kmeans.fit(load_data[:, :int(load_data.shape[1]*train_ratio)])
    cluster_list = np.random.randint(low=0, high=15, size=(1000,))
    process_list.append(cluster_list)
    for epoch in range(max_epoch):
        original_list = cluster_list
        aggr_data = aggregate_by_cluster_list(load=load_data,labels=cluster_list,num_cluster=K, agg='mean')
        # 2.MLR预测模型构建 Forecasting Models Training
        model_dict = train_model(cluster_num=K, cluster_list=cluster_list, aggr_data=aggr_data, 
                                lag=24,train_ratio=train_ratio, Radiation=Radiation,Calendar=Calendar)
        # 3.构建误差反馈矩阵 Feedback Mechanism
        matrix = error_feedback(load_data=load_data,cluster_num=K, model_dict=model_dict,lag=24,
                                train_ratio=train_ratio, val_ratio=val_ratio, Radiation=Radiation,Calendar=Calendar)
        # 4.更新簇标签 Updating Clustering Membership
        cluster_list = np.argmin(matrix,axis=1)
        change_num = change_rate_simple(old_labels=original_list,new_labels=cluster_list)
        process_list.append(cluster_list)
        if change_num <= 1:
            print("finish")
            break
        else:
            # pass
            print('Change Num =',change_num)
    predict_result, test_result = Predict(cluster_num=K,cluster_list=cluster_list,
                                        aggr_data=aggr_data,lag=24,train_ratio=train_ratio,
                                        val_ratio=val_ratio,Radiation=Radiation,
                                        Calendar=Calendar,model_dict=model_dict)
    unique_elements = np.unique(cluster_list)

    user_belong = count_clusters(num_cluster=K, cluster_list=cluster_list)
    for index, cluster_id in enumerate(unique_elements):
            predict_result[index] = predict_result[index]*len(user_belong[str(cluster_id)])
            test_result[index] = test_result[index]*len(user_belong[str(cluster_id)])
    predict_result = predict_result.sum(axis=0)
    test_result = test_result.sum(axis=0)

    # np.save(save_path+"predict_result.npy",predict_result)
    # np.save(save_path+"test_result.npy",test_result)

    MAE  = np.mean(np.abs(predict_result-test_result))
    MAPE = np.mean(np.abs(predict_result-test_result)/test_result)*100
    
    MAE_list.append(MAE)
    MAPE_list.append(MAPE)
    print("Epoch:",K,"FINISHED")
    
    
    num = 0
    for index in range(K):
        if model_dict[str(index)] == None:
            pass
        else:
            num +=1
    print("Final Cluster Num = ",num,"\n")
    
MAPE_list = np.asarray(MAPE_list)
MAE_list = np.asarray(MAE_list)

print("最小MAE聚类数",np.argmin(MAE_list)+1,"最小MAE",np.min(MAE_list))
print("最小MAPE聚类数",np.argmin(MAPE_list)+1,"最小MAPE",np.min(MAPE_list))

print(count_clusters(num_cluster=K, cluster_list=cluster_list))

Change Num = 918
Change Num = 229
Change Num = 107
Change Num = 80
Change Num = 66
Change Num = 45
Change Num = 33
Change Num = 28
Change Num = 16
Change Num = 8
Change Num = 9
Change Num = 21
Change Num = 20
Change Num = 20
Change Num = 7
Change Num = 4
finish
Epoch: 15 FINISHED
Final Cluster Num =  7 

最小MAE聚类数 1 最小MAE 131.33375192873632
最小MAPE聚类数 1 最小MAPE 3.707158479861964
{'0': [], '1': [], '2': [7, 18, 64, 94, 100, 119, 140, 151, 152, 187, 196, 201, 206, 221, 226, 233, 238, 249, 287, 303, 323, 331, 339, 364, 378, 384, 408, 419, 468, 478, 521, 530, 546, 570, 607, 627, 629, 630, 640, 641, 713, 729, 743, 744, 760, 793, 807, 814, 817, 821, 858, 860, 866, 876, 905, 951, 979, 998], '3': [], '4': [], '5': [400, 464], '6': [0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 65, 66, 67, 69, 70, 71, 72, 73, 74

In [4]:
process_list = np.asarray(process_list)
print(process_list.shape)

(18, 1000)


In [19]:
import numpy as np
import matplotlib.pyplot as plt
import imageio.v2 as imageio
import io

# ====== 参数配置 ======
num_interp = 5  # 每两个epoch之间插入多少中间帧（越大越平滑）
frames = []

# ====== 生成动画帧 ======
for i in range(process_list.shape[0] - 1):
    counts_start = np.bincount(process_list[i], minlength=15)
    counts_end   = np.bincount(process_list[i + 1], minlength=15)

    # 在两个分布之间插值
    for t in np.linspace(0, 1, num_interp, endpoint=False):
        counts_interp = counts_start * (1 - t) + counts_end * t

        # 绘图
        fig, ax = plt.subplots(figsize=(6, 4), dpi=150)
        bars = ax.bar(range(1, 16), counts_interp, color='skyblue')
        ax.set_yscale('log')
        ax.set_title(f'Epoch {i+1} Trim & Merge')
        ax.set_xlabel('Cluster ID')
        ax.set_ylabel('log(User Amount)')
        ax.set_xticks(range(1, 16))
        plt.tight_layout()

        # 在柱子上添加数值标签
        for rect, c in zip(bars, counts_interp):
            if c > 0:
                ax.text(
                    rect.get_x() + rect.get_width()/2,
                    max(rect.get_height(), 1e-1),
                    f'{int(c)}',
                    ha='center', va='bottom', fontsize=7
                )

        # 转为内存图像帧
        buf = io.BytesIO()
        plt.savefig(buf, format='png', dpi=150)
        buf.seek(0)
        frames.append(imageio.imread(buf))
        plt.close(fig)

# 最后一帧：完整显示最终分布
fig, ax = plt.subplots(figsize=(6, 4))
counts_final = np.bincount(process_list[-1], minlength=15)
bars = ax.bar(range(1,16), counts_final, color='skyblue')
ax.set_yscale('log')
ax.set_title(f'Final Epoch {process_list.shape[0]} Trim & Merge')
ax.set_xlabel('Cluster ID')
ax.set_ylabel('log(User Amount)')
ax.set_xticks(range(1,16))
plt.tight_layout()

for rect, c in zip(bars, counts_final):
    if c > 0:
        ax.text(
            rect.get_x() + rect.get_width()/2,
            max(rect.get_height(), 1e-1),
            f'{int(c)}',
            ha='center', va='bottom', fontsize=7
        )

buf = io.BytesIO()
plt.savefig(buf, format='png', dpi=150)
buf.seek(0)
frames.append(imageio.imread(buf))
plt.close(fig)

# ====== 保存 GIF 动画 ======
imageio.mimsave('Trim & Merge.gif', frames, fps=10)
print("✅ 已生成带平滑过渡的 GIF：Trim & Merge.gif")

✅ 已生成带平滑过渡的 GIF：Trim & Merge.gif


In [6]:
# import numpy as np
# import matplotlib.pyplot as plt
# import imageio.v2 as imageio  # imageio.v2 兼容新版写法
# import os

# # 创建临时文件夹保存帧
# os.makedirs("frames", exist_ok=True)

# # 绘制每一帧
# for i in range(process_list.shape[0]):
#     counts = np.bincount(process_list[i], minlength=16)  # 注意要16，不然少最后一列

#     plt.figure(figsize=(6, 4),dpi=150)
#     bars = plt.bar(range(16), counts, color='skyblue')
#     plt.yscale('log')  # 使用对数坐标
#     plt.title(f'Epoch {i+1} Trim & Merge')
#     plt.xlabel('Value (0–15)')
#     plt.ylabel('log(Count)')
#     plt.xticks(range(16))
#     plt.tight_layout()

#     # ⭐ 在每根柱子上标出数值
#     for rect, c in zip(bars, counts):
#         height = rect.get_height()
#         # 如果高度太小可以略微调整显示位置
#         if c > 0:
#             plt.text(
#                 rect.get_x() + rect.get_width()/2,  # x 位置：柱子中心
#                 height,                              # y 位置：柱顶
#                 f'{c}',                              # 显示实际数值
#                 ha='center', va='bottom', fontsize=10, rotation=0
#             )
            
#     # 保存这一帧
#     filename = f"frames/frame_{i:03d}.png"
#     plt.savefig(filename)
#     plt.close()  # 关闭图像防止内存泄漏

# # 读取所有帧并生成GIF
# frames = []
# for i in range(process_list.shape[0]):
#     filename = f"frames/frame_{i:03d}.png"
#     frames.append(imageio.imread(filename))
#     frames.append(imageio.imread(filename))
#     frames.append(imageio.imread(filename))

# # 保存为gif，duration控制每帧时长（秒）
# imageio.mimsave('Trim & Merge.gif', frames, fps=8)

# print("✅ 动图已保存为 Trim & Merge.gif")
